--- Day 4: Scratchcards ---

The gondola takes you up. Strangely, though, the ground doesn't seem to be coming with you; you're not climbing a mountain. As the circle of Snow Island recedes below you, an entire new landmass suddenly appears above you! The gondola carries you to the surface of the new island and lurches into the station.

As you exit the gondola, the first thing you notice is that the air here is much warmer than it was on Snow Island. It's also quite humid. Is this where the water source is?

The next thing you notice is an Elf sitting on the floor across the station in what seems to be a pile of colorful square cards.

"Oh! Hello!" The Elf excitedly runs over to you. "How may I be of service?" You ask about water sources.

"I'm not sure; I just operate the gondola lift. That does sound like something we'd have, though - this is Island Island, after all! I bet the gardener would know. He's on a different island, though - er, the small kind surrounded by water, not the floating kind. We really need to come up with a better naming scheme. Tell you what: if you can help me with something quick, I'll let you borrow my boat and you can go visit the gardener. I got all these scratchcards as a gift, but I can't figure out what I've won."

The Elf leads you over to the pile of colorful cards. There, you discover dozens of scratchcards, all with their opaque covering already scratched off. Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).

As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.

For example:

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53

Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19

Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1

Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83

Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36

Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

In the above example, card 1 has five winning numbers (41, 48, 83, 86, and 17) and eight numbers you have (83, 86, 6, 31, 17, 9, 48, and 53). Of the numbers you have, four of them (48, 83, 17, and 86) are winning numbers! That means card 1 is worth 8 points (1 for the first match, then doubled three times for each of the three matches after the first).

Card 2 has two winning numbers (32 and 61), so it is worth 2 points.
Card 3 has two winning numbers (1 and 21), so it is worth 2 points.
Card 4 has one winning number (84), so it is worth 1 point.
Card 5 has no winning numbers, so it is worth no points.
Card 6 has no winning numbers, so it is worth no points.
So, in this example, the Elf's pile of scratchcards is worth 13 points.

Take a seat in the large pile of colorful cards. How many points are they worth in total?

In [1]:
test_input = """
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"""

In [2]:
from __future__ import annotations
from dataclasses import dataclass


@dataclass
class ScratchCard:
    game_number: int
    winner_values: set[int]
    player_values: set[int]

    @classmethod
    def from_string(cls, scrachcard_str: str) -> ScratchCard:
        # example string: "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"
        game_number_str, values_str = scrachcard_str.split(":")
        game_number = int(game_number_str.strip().split()[-1])
        winner_values_str, player_values_str = values_str.strip().split("|")
        winner_values = set(int(x) for x in winner_values_str.strip().split())
        player_values = set(int(x) for x in player_values_str.strip().split())
        return cls(game_number=game_number, winner_values=winner_values, player_values=player_values)
    
    @property
    def number_matches(self) -> int:
        player_winner_values = self.winner_values.intersection(self.player_values)
        return len(player_winner_values)


    @property
    def card_value(self) -> int:
        if self.number_matches:
            return 2 ** (self.number_matches - 1)
        else:
            return 0


def update_int_counter_map(count_map: dict[int, int], value: int, n_times: int = 1):
    if value in count_map:
        count_map[value] += n_times
    else:
        count_map[value] = n_times


@dataclass
class CardPile:
    scratchcards: dict[int, ScratchCard]

    @classmethod
    def from_string(cls, cardpile_str: str) -> CardPile:
        scratchcard_str_list = cardpile_str.strip().split("\n")
        scratchcards = {}
        for sc_str in scratchcard_str_list:
            sc = ScratchCard.from_string(sc_str)
            scratchcards[sc.game_number] = sc
        return cls(scratchcards = scratchcards)
    
    @property
    def total_score(self) -> int:
        return sum([sc.card_value for sc in self.scratchcards.values()])

    @property
    def scratchcard_count_game(self) -> int:
        # count of cards when playing the game in part 2
        card_bank = {}
        max_game_number = max(self.scratchcards.keys())
        for game_number in sorted(self.scratchcards.keys()):
            sc = self.scratchcards[game_number]
            this_card_count_in_bank = card_bank.get(game_number, 0)
            number_new_cards = sc.number_matches
            for next_game_number in range(game_number + 1, min(max_game_number + 1, game_number + 1 + number_new_cards)):
                update_int_counter_map(card_bank, next_game_number, (1 + this_card_count_in_bank))

        return sum(card_bank.values()) + len(self.scratchcards)


In [3]:
testpile = CardPile.from_string(test_input)

assert(testpile.total_score) == 13

In [4]:
with open("../inputs/day4_input.txt") as f:
    input_str = f.read()


In [5]:
pile = CardPile.from_string(input_str)

pile.total_score

22674

# Part 2

--- Part Two ---
Just as you're about to report your findings to the Elf, one of you realizes that the rules have actually been printed on the back of every card this whole time.

There's no such thing as "points". Instead, scratchcards only cause you to win more scratchcards equal to the number of winning numbers you have.

Specifically, you win copies of the scratchcards below the winning card equal to the number of matches. So, if card 10 were to have 5 matching numbers, you would win one copy each of cards 11, 12, 13, 14, and 15.

Copies of scratchcards are scored like normal scratchcards and have the same card number as the card they copied. So, if you win a copy of card 10 and it has 5 matching numbers, it would then win a copy of the same cards that the original card 10 won: cards 11, 12, 13, 14, and 15. This process repeats until none of the copies cause you to win any more cards. (Cards will never make you copy a card past the end of the table.)

This time, the above example goes differently:

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53

Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19

Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1

Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83

Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36

Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

Card 1 has four matching numbers, so you win one copy each of the next four cards: cards 2, 3, 4, and 5.
Your original card 2 has two matching numbers, so you win one copy each of cards 3 and 4.
Your copy of card 2 also wins one copy each of cards 3 and 4.
Your four instances of card 3 (one original and three copies) have two matching numbers, so you win four copies each of cards 4 and 5.
Your eight instances of card 4 (one original and seven copies) have one matching number, so you win eight copies of card 5.
Your fourteen instances of card 5 (one original and thirteen copies) have no matching numbers and win no more cards.
Your one instance of card 6 (one original) has no matching numbers and wins no more cards.
Once all of the originals and copies have been processed, you end up with 1 instance of card 1, 2 instances of card 2, 4 instances of card 3, 8 instances of card 4, 14 instances of card 5, and 1 instance of card 6. In total, this example pile of scratchcards causes you to ultimately have 30 scratchcards!

Process all of the original and copied scratchcards until no more scratchcards are won. Including the original set of scratchcards, how many total scratchcards do you end up with?



In [6]:
test_input_2 = """
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"""

testpile_2 = CardPile.from_string(test_input_2)



In [7]:
testpile_2

CardPile(scratchcards={1: ScratchCard(game_number=1, winner_values={41, 48, 17, 83, 86}, player_values={6, 9, 48, 17, 83, 53, 86, 31}), 2: ScratchCard(game_number=2, winner_values={32, 13, 16, 20, 61}, player_values={32, 68, 17, 82, 19, 24, 61, 30}), 3: ScratchCard(game_number=3, winner_values={1, 44, 53, 21, 59}, player_values={1, 69, 72, 14, 16, 82, 21, 63}), 4: ScratchCard(game_number=4, winner_values={69, 73, 41, 84, 92}, player_values={5, 76, 51, 84, 83, 54, 58, 59}), 5: ScratchCard(game_number=5, winner_values={32, 83, 87, 26, 28}, player_values={36, 70, 12, 82, 22, 88, 93, 30}), 6: ScratchCard(game_number=6, winner_values={72, 13, 18, 56, 31}, player_values={35, 67, 36, 74, 10, 11, 77, 23})})

In [8]:
assert testpile_2.scratchcard_count_game == 30

In [9]:
pile.scratchcard_count_game

5747443